In [1]:
# calculate parallel corpus size and DM freqs in it, based on FILESOURCES, instead of props
# I failed to turn elements of code into functions; the problem is I am unable to return all results in one piece
import re
import sys, codecs
from xml.dom import minidom
import numpy as np
import scipy.stats as stats
import csv

In [2]:
arg1 = '/home/masha/birmingham/data/prof_tagged.tmx' #sys.argv[1]# this is a clean and tidy rltc_tmx! it does not work for prof
arg2 = '/home/masha/birmingham/bi-ling_EMs.ls' #sys.argv[2]#список поисковых запросов 

In [3]:
doc = minidom.parse(arg1)
errors = 0
restore_texts = {}

tus = doc.getElementsByTagName("tu") #возвращает список tu (элемента, содержащего tuvs)

In [4]:
# loop over all tuvs in tus to create a list of keys in a dic; this dic has a list of values for each key
fns = []
for tu in tus:
    tuvs = tu.getElementsByTagName("tuv")
    for tuv in tuvs:
        fn = tuv.getAttribute('filesource')
        if len(fn) == 0:
            error += 1
        fn = fn.strip()
        if fn not in fns:
            fns.append(fn)
statistics = {fn :{} for fn in fns}
for key in statistics.keys():
    print key
        

NameError: name 'error' is not defined

In [ ]:
# build the dic and collect all values
print len(tus)

for tu in tus:
    tuvs = tu.getElementsByTagName("tuv")
    for tuv in tuvs:
        fn = tuv.getAttribute('filesource')
        if len(fn) == 0:
            error += 1
        fn = fn.strip()
        seg_el = tuv.getElementsByTagName("seg")[0]
        seg_text = seg_el.childNodes[-1].data
        seg_text = seg_text.strip()
        
        try:
            restore_texts[fn].append(seg_text)
        except KeyError:
            restore_texts[fn] = [seg_text]
print errors       


In [ ]:
# writing and printing a dic is a trick

for fn, segs in restore_texts.items(): # or .iteritems()
    print fn, '\t', len(segs)
    try:
        statistics[fn]['tuvs'].append(len(segs))
    except KeyError:
        statistics[fn]['tuvs'] = len(segs)

In [ ]:
# incrementing the amount of results
    
for fn, nest in statistics.iteritems():
    print fn, '\t', nest['tuvs']


In [ ]:
# produce word count and number of sentences for each file, which can be accessed as restore_segs[fn]

for fn, segs in restore_texts.items():
    num_SENTs = 0
    wc = 0
    for seg in segs:
        seg = seg.replace('?_SENT_?', '._SENT_.')
        seg = seg.replace('!_SENT_!', '._SENT_.')
        seg = seg.replace('._SENT_.', '._SENT_.')
        seg = seg.replace('._SENT_.', '._SENT_.')
        alt_SENTs = seg.count('._SENT_.')
        words = len(seg.split())
        num_SENTs += alt_SENTs
        wc += words

    print fn, '\t', wc, '\t', num_SENTs, '\n'
    statistics[fn]['SENTs'] = num_SENTs
    statistics[fn]['wc'] = wc

In [ ]:
# incrementing the amount of results
for fn, nest in statistics.iteritems():
    print fn, '\t', nest['tuvs'], '\t', nest['SENTs'], '\t', nest['wc']

In [ ]:
# count freqs of tagged items - CONNs and EMs
    
queries = codecs.open(arg2, 'r', 'utf-8').readlines()

for fn, segs in restore_texts.items():
    count_item = 0
    for seg in segs:
        for item in queries:
            item=item.strip()
            #print item
            freq = seg.find(item)
            if -1==freq: # if conn is nothing
                continue
            else: 
            #print fn, item
                count_item +=1   
    print fn, '\t', count_item
    statistics[fn]['freq'] = count_item

In [ ]:
# print results from the nested dict of {fn : {'tuv' : 2, 'SENTs' : 2, 'wc' : 37, 'freq' : 4}}

for fn, nest in statistics.iteritems():
    print fn, '\t', nest['tuvs'], '\t', nest['SENTs'], '\t', nest['wc'], '\t', nest['freq']

In [ ]:
# запишем-ка в файл
with open("/home/masha/birmingham/stats/prof_tagged.csv", 'w') as outfile: #adjust outfile names appropriately
    writer = csv.writer(outfile, delimiter='\t')
    for fn, nest in statistics.iteritems():
        writer.writerow([fn.encode('utf-8')] + [nest['tuvs']] + [nest['SENTs']] + [nest['wc']] + [nest['freq']])  